<a href="https://colab.research.google.com/github/slxslxslx/BiShe/blob/main/sentiment140_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# 尝试-这个分布式可以顺利运行
# ds = load_dataset("contemmcm/sentiment140")
# 百万级别1600000，二分类
# 训练集数据量: DatasetDict({
#     complete: Dataset({
#         features: ['text', 'label'],
#         num_rows: 1600000
#     })
# })
# RTX 2080 Ti * 2
# Single GPU training time: 212.33 seconds
#    accuracy                           0.88
# 多GPU,rank=1 end_timeMulti GPU training time: 149.43 seconds
#     accuracy                           0.88
import os

# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

import time
from datetime import datetime, timedelta, timezone
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split, DistributedSampler
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

from sklearn.metrics import classification_report
import numpy as np
import pickle

# 获取北京时区对象
beijing_tz = timezone(timedelta(hours=8))


# 获取当前的UTC时间并转换为北京时间
def get_beijing_time_now():
    return datetime.now(beijing_tz).strftime("%Y-%m-%d %H:%M:%S")


# 加载IMDB数据集和BERT分词器，并进行一些初始化操作
def load_and_preprocess_data():
    # print('load_and_preprocess_data开始执行')
    # dataset = load_dataset("contemmcm/sentiment140", cache_dir="./dataset/sentiment140")
    # print("dataset['complete'][:5]:", dataset['complete'][:5])
    # tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # # 文本编码函数
    # def encode_batch(batch):
    #     return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

    # # 编码数据集
    # print('开始编码数据集',get_beijing_time_now())
    # encoded_dataset = dataset.map(encode_batch, batched=True)
    # encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    # print("encoded_dataset['complete'][:5]:", encoded_dataset['complete'][:5])
    # print('结束编码数据集',get_beijing_time_now())

    # # 保存encoded_dataset
    # with open('encoded_dataset-sentiment140.pkl','wb') as f:
    #     pickle.dump(encoded_dataset, f)
    #     print('保存encoded_dataset-sentiment140.pkl')

    with open('encoded_dataset-sentiment140.pkl', 'rb') as f:
        encoded_dataset = pickle.load(f)

    # 划分训练集和测试集
    # 假设encoded_dataset['complete']是你的数据集，这里获取其长度
    total_length = len(encoded_dataset['complete'])
    # 计算训练集的大小（这里按80%比例计算，可根据实际需求修改比例）
    train_size = int(total_length * 0.8)
    # 计算测试集的大小
    test_size = total_length - train_size
    print('total_length', total_length)
    print('train_size', train_size)
    print('test_size', test_size)
    # train_dataset, test_dataset = random_split(encoded_dataset['complete'], [20000, 5000])
    train_dataset, test_dataset = random_split(encoded_dataset['complete'], [train_size, test_size])

    # 使用数据集中自带的train部分作为训练集
    # train_dataset = encoded_dataset['train']
    # test_dataset = encoded_dataset['test']

    test_loader = DataLoader(test_dataset, batch_size=16)
    print('load_and_preprocess_data执行完毕')
    return train_dataset, test_dataset, test_loader


# 单GPU训练
def train_single_gpu(model, train_dataset, epochs=1):
    print('单GPU训练开始,epochs=', epochs)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=1e-5)  # 优化器用于根据计算得到的损失值来更新模型的参数
    # 以使得模型能够朝着损失值不断减小的方向进行优化，进而提高模型对数据的拟合能力和预测准确性。
    # 自适应矩估计（Adaptive Moment Estimation）优化算法
    # model.parameters() 获取模型中所有需要被优化的参数

    start_time = time.time()
    print('单GPU start_time', get_beijing_time_now())
    model.train()
    for epoch in range(epochs):
        print('epoch', epoch)
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

    end_time = time.time()
    print('单GPU end_time', get_beijing_time_now())
    print(f"Single GPU training time: {end_time - start_time:.2f} seconds")

    # 评估模型
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():  # 不进行梯度计算
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print('单GPU评估:\n', classification_report(all_labels, all_preds))


# 2GPU训练
def train_multi_gpu(rank, world_size, model, train_dataset, test_loader, epochs=1):
    # print('test_loader', test_loader)
    print(f"多GPU分布式训练进程 {rank} 初始化")

    # 设置分布式环境变量
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size, timeout=timedelta(seconds=180))

    # 设定设备
    device = torch.device(f"cuda:{rank}")
    model = model.to(device)
    model = DDP(model, device_ids=[rank])
    # device_ids=[rank] 这个参数就是明确告诉 DDP 模块，当前这个模型副本应该放置在哪个具体的 GPU 设备上进行后续的训练操作。保证每个进程所负责的模型副本和相应的 GPU 紧密关联起来

    # 使用 DistributedSampler 确保每个进程处理不同的数据
    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, sampler=train_sampler)
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    criterion = nn.CrossEntropyLoss()

    start_time = time.time()
    print(f'多GPU,rank={rank} start_time', get_beijing_time_now())
    if rank == 0:
        print(f"多GPU训练开始时间: {get_beijing_time_now()}")
    model.train()

    for epoch in range(epochs):
        if rank == 0:
            print(f"Epoch {epoch} 开始")
        train_sampler.set_epoch(epoch)
        for batch in train_loader:
            # print(rank, 'batch')
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

    end_time = time.time()
    print(f'多GPU,rank={rank} end_time', get_beijing_time_now())
    if rank == 0:
        print(f"Multi GPU training time: {end_time - start_time:.2f} seconds")

    # 评估模型
    if rank == 0:
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        print('多GPU评估:\n', classification_report(all_labels, all_preds))

    dist.destroy_process_group()


# 设置进程启动，通过mp.spawn启动多进程分布式训练
def spawn_train(world_size, model, train_dataset, test_loader):
    mp.spawn(train_multi_gpu, args=(world_size, model, train_dataset, test_loader), nprocs=world_size,
             join=True)  # join=True时mp.spawn 函数会阻塞主进程，直到所有启动的子进程都执行完毕。


# 主函数入口
if __name__ == '__main__':
    train_dataset, test_dataset, test_loader = load_and_preprocess_data()
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数

    # 单GPU训练
    train_single_gpu(model, train_dataset)

    # 多GPU分布式训练!
    # spawn_train(world_size=2, model=model, train_dataset=train_dataset, test_loader=test_loader)

    print('main')


FileNotFoundError: [Errno 2] Failed to open local file '/content/dataset/sentiment140/contemmcm___sentiment140/default/0.0.0/3fbbc6d5cab2192c8cb7591bdf5b1d4360ea74fd/cache-7ee2a0e947c4ccf8.arrow'. Detail: [errno 2] No such file or directory

In [ ]:
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

import time
from datetime import datetime, timedelta, timezone
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split, DistributedSampler
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

from sklearn.metrics import classification_report
from torch.cuda.amp import GradScaler, autocast  # 【新增】支持混合精度训练
from torch.utils.tensorboard import SummaryWriter  # 【新增】支持日志记录

import pickle

# 获取北京时区对象
beijing_tz = timezone(timedelta(hours=8))


# 获取当前的UTC时间并转换为北京时间
def get_beijing_time_now():
    return datetime.now(beijing_tz).strftime("%Y-%m-%d %H:%M:%S")


# 加载IMDB数据集和BERT分词器，并进行一些初始化操作
def load_and_preprocess_data():
    print('load_and_preprocess_data开始执行')
    dataset = load_dataset("contemmcm/sentiment140", cache_dir="./dataset/sentiment140")
    print("dataset['complete'][:5]:", dataset['complete'][:5])
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # # 文本编码函数
    # def encode_batch(batch):
    #     return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

    # # 编码数据集
    # print('开始编码数据集', get_beijing_time_now())
    # encoded_dataset = dataset.map(encode_batch, batched=True)
    # encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    # print("encoded_dataset['complete'][:5]:", encoded_dataset['complete'][:5])
    # print('结束编码数据集', get_beijing_time_now())

    with open('encoded_dataset-sentiment140.pkl', 'rb') as f:
        encoded_dataset = pickle.load(f)
        print('加载encoded_dataset-sentiment140.pkl完毕')

    # 划分训练集和测试集
    total_length = len(encoded_dataset['complete'])
    train_size = int(total_length * 0.8)
    test_size = total_length - train_size
    train_dataset, test_dataset = random_split(encoded_dataset['complete'], [train_size, test_size])

    print('total_length', total_length)
    print('train_size', train_size)
    print('test_size', test_size)

    print('load_and_preprocess_data执行完毕')
    return train_dataset, test_dataset


# 单GPU训练
def train_single_gpu(model, train_dataset, test_dataset, epochs=3):
    print('单GPU训练开始,epochs=', epochs)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    batch_size = 64  # 【修改】适配更大的 batch_size，充分利用显存
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # 【修改】调高学习率
    criterion = nn.CrossEntropyLoss()  # 损失函数
    scaler = GradScaler()  # 【新增】混合精度训练支持
    writer = SummaryWriter(log_dir='./logs')  # 【新增】日志记录

    start_time = time.time()
    print('单GPU start_time', get_beijing_time_now())
    model.train()
    for epoch in range(epochs):
        print('Epoch', epoch)
        epoch_loss = 0.0

        for step, batch in enumerate(train_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            with autocast():  # 【新增】混合精度上下文
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = criterion(outputs.logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()

        print(f"Epoch {epoch}: Loss = {epoch_loss / len(train_loader):.4f}")
        writer.add_scalar("Loss/train", epoch_loss / len(train_loader), epoch)

    end_time = time.time()
    print('单GPU end_time', get_beijing_time_now())
    print(f"Single GPU training time: {end_time - start_time:.2f} seconds")

    # 评估模型
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print('单GPU评估:\n', classification_report(all_labels, all_preds))


# 2GPU分布式训练
def train_multi_gpu(rank, world_size, model, train_dataset, test_dataset, epochs=3):
    print(f"多GPU分布式训练进程 {rank} 初始化")

    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size, timeout=timedelta(seconds=180))

    device = torch.device(f"cuda:{rank}")
    model = model.to(device)
    model = DDP(model, device_ids=[rank])

    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    train_loader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler, num_workers=4, pin_memory=True)

    test_sampler = DistributedSampler(test_dataset, num_replicas=world_size, rank=rank, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=64, sampler=test_sampler, num_workers=4, pin_memory=True)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()

    start_time = time.time()
    model.train()

    for epoch in range(epochs):
        train_sampler.set_epoch(epoch)
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            with autocast():
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = criterion(outputs.logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

    end_time = time.time()
    if rank == 0:
        print(f"Multi GPU training time: {end_time - start_time:.2f} seconds")

    dist.destroy_process_group()


# 启动分布式训练
def spawn_train(world_size, model, train_dataset, test_dataset):
    mp.spawn(train_multi_gpu, args=(world_size, model, train_dataset, test_dataset), nprocs=world_size, join=True)


if __name__ == '__main__':
    train_dataset, test_dataset = load_and_preprocess_data()
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    # 单GPU训练
    train_single_gpu(model, train_dataset, test_dataset)

    # 多GPU训练
    # spawn_train(world_size=2, model=model, train_dataset=train_dataset, test_dataset=test_dataset)

load_and_preprocess_data开始执行
dataset['complete'][:5]: {'text': ["Just back home from a little gathering with some old friends.. It was really fun, they're still the same. ", 'Hey @ricebunny i need a web cam!!!   (RiceBunny live > http://ustre.am/ZbT)', 'only a couple more days before i have to go to faggot summer school ', 'Must admit Zac handled it well!!! ', "It's warm and sticky but there's no sun  *Sneeze*"], 'label': [1, 0, 0, 0, 0]}


FileNotFoundError: [Errno 2] Failed to open local file '/content/dataset/sentiment140/contemmcm___sentiment140/default/0.0.0/3fbbc6d5cab2192c8cb7591bdf5b1d4360ea74fd/cache-7ee2a0e947c4ccf8.arrow'. Detail: [errno 2] No such file or directory

In [ ]:
# 尝试-这个分布式可以顺利运行
# ds = load_dataset("contemmcm/sentiment140")
# 百万级别1600000，二分类
# 训练集数据量: DatasetDict({
#     complete: Dataset({
#         features: ['text', 'label'],
#         num_rows: 1600000
#     })
# })
# RTX 2080 Ti * 2
# Single GPU training time: 212.33 seconds
#    accuracy                           0.88
# 多GPU,rank=1 end_timeMulti GPU training time: 149.43 seconds
#     accuracy                           0.88
import os

# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

import time
from datetime import datetime, timedelta, timezone
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split, DistributedSampler
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

from sklearn.metrics import classification_report
import numpy as np
import pickle

# 获取北京时区对象
beijing_tz = timezone(timedelta(hours=8))


# 获取当前的UTC时间并转换为北京时间
def get_beijing_time_now():
    return datetime.now(beijing_tz).strftime("%Y-%m-%d %H:%M:%S")


# 加载IMDB数据集和BERT分词器，并进行一些初始化操作
def load_and_preprocess_data():
    # print('load_and_preprocess_data开始执行')
    # dataset = load_dataset("contemmcm/sentiment140", cache_dir="./dataset/sentiment140")
    # print("dataset['complete'][:5]:", dataset['complete'][:5])
    # tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # # 文本编码函数
    # def encode_batch(batch):
    #     return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

    # # 编码数据集
    # print('开始编码数据集',get_beijing_time_now())
    # encoded_dataset = dataset.map(encode_batch, batched=True)
    # encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    # print("encoded_dataset['complete'][:5]:", encoded_dataset['complete'][:5])
    # print('结束编码数据集',get_beijing_time_now())

    # # 保存encoded_dataset
    # with open('encoded_dataset-sentiment140.pkl','wb') as f:
    #     pickle.dump(encoded_dataset, f)
    #     print('保存encoded_dataset-sentiment140.pkl')

    with open('encoded_dataset-sentiment140.pkl', 'rb') as f:
        encoded_dataset = pickle.load(f)

    # 划分训练集和测试集
    # 假设encoded_dataset['complete']是你的数据集，这里获取其长度
    total_length = len(encoded_dataset['complete'])
    # 计算训练集的大小（这里按80%比例计算，可根据实际需求修改比例）
    train_size = int(total_length * 0.8)
    # 计算测试集的大小
    test_size = total_length - train_size
    print('total_length', total_length)
    print('train_size', train_size)
    print('test_size', test_size)
    # train_dataset, test_dataset = random_split(encoded_dataset['complete'], [20000, 5000])
    train_dataset, test_dataset = random_split(encoded_dataset['complete'], [train_size, test_size])

    # 使用数据集中自带的train部分作为训练集
    # train_dataset = encoded_dataset['train']
    # test_dataset = encoded_dataset['test']

    test_loader = DataLoader(test_dataset, batch_size=16)
    print('load_and_preprocess_data执行完毕')
    return train_dataset, test_dataset, test_loader


# 单GPU训练
def train_single_gpu(model, train_dataset, epochs=1):
    print('单GPU训练开始,epochs=', epochs)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=1e-5)  # 优化器用于根据计算得到的损失值来更新模型的参数
    # 以使得模型能够朝着损失值不断减小的方向进行优化，进而提高模型对数据的拟合能力和预测准确性。
    # 自适应矩估计（Adaptive Moment Estimation）优化算法
    # model.parameters() 获取模型中所有需要被优化的参数

    start_time = time.time()
    print('单GPU start_time', get_beijing_time_now())
    model.train()
    for epoch in range(epochs):
        print('epoch', epoch)
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

    end_time = time.time()
    print('单GPU end_time', get_beijing_time_now())
    print(f"Single GPU training time: {end_time - start_time:.2f} seconds")

    # 评估模型
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():  # 不进行梯度计算
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print('单GPU评估:\n', classification_report(all_labels, all_preds))


# 2GPU训练
def train_multi_gpu(rank, world_size, model, train_dataset, test_loader, epochs=1):
    # print('test_loader', test_loader)
    print(f"多GPU分布式训练进程 {rank} 初始化")

    # 设置分布式环境变量
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size, timeout=timedelta(seconds=180))

    # 设定设备
    device = torch.device(f"cuda:{rank}")
    model = model.to(device)
    model = DDP(model, device_ids=[rank])
    # device_ids=[rank] 这个参数就是明确告诉 DDP 模块，当前这个模型副本应该放置在哪个具体的 GPU 设备上进行后续的训练操作。保证每个进程所负责的模型副本和相应的 GPU 紧密关联起来

    # 使用 DistributedSampler 确保每个进程处理不同的数据
    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, sampler=train_sampler)
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    criterion = nn.CrossEntropyLoss()

    start_time = time.time()
    print(f'多GPU,rank={rank} start_time', get_beijing_time_now())
    if rank == 0:
        print(f"多GPU训练开始时间: {get_beijing_time_now()}")
    model.train()

    for epoch in range(epochs):
        if rank == 0:
            print(f"Epoch {epoch} 开始")
        train_sampler.set_epoch(epoch)
        for batch in train_loader:
            # print(rank, 'batch')
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

    end_time = time.time()
    print(f'多GPU,rank={rank} end_time', get_beijing_time_now())
    if rank == 0:
        print(f"Multi GPU training time: {end_time - start_time:.2f} seconds")

    # 评估模型
    if rank == 0:
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        print('多GPU评估:\n', classification_report(all_labels, all_preds))

    dist.destroy_process_group()


# 设置进程启动，通过mp.spawn启动多进程分布式训练
def spawn_train(world_size, model, train_dataset, test_loader):
    mp.spawn(train_multi_gpu, args=(world_size, model, train_dataset, test_loader), nprocs=world_size,
             join=True)  # join=True时mp.spawn 函数会阻塞主进程，直到所有启动的子进程都执行完毕。


# 主函数入口
if __name__ == '__main__':
    train_dataset, test_dataset, test_loader = load_and_preprocess_data()
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数

    # 单GPU训练
    train_single_gpu(model, train_dataset)

    # 多GPU分布式训练!
    # spawn_train(world_size=2, model=model, train_dataset=train_dataset, test_loader=test_loader)

    print('main')


FileNotFoundError: [Errno 2] Failed to open local file '/content/dataset/sentiment140/contemmcm___sentiment140/default/0.0.0/3fbbc6d5cab2192c8cb7591bdf5b1d4360ea74fd/cache-7ee2a0e947c4ccf8.arrow'. Detail: [errno 2] No such file or directory

In [ ]:
chmod +w /content/dataset/sentiment140

NameError: name 'chmod' is not defined